<a href="https://www.kaggle.com/code/raxmatillo/aviachiptabashorat?scriptVersionId=218912022" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aviachipta-narxini-bashorat-qilish/train_data.csv
/kaggle/input/aviachipta-narxini-bashorat-qilish/test_data.csv
/kaggle/input/aviachipta-narxini-bashorat-qilish/sample_solution.csv


# Ma'lumotlarni tayyorlab olamiz

In [2]:
train = pd.read_csv('/kaggle/input/aviachipta-narxini-bashorat-qilish/train_data.csv')
test = pd.read_csv('/kaggle/input/aviachipta-narxini-bashorat-qilish/test_data.csv')
sample = pd.read_csv('/kaggle/input/aviachipta-narxini-bashorat-qilish/sample_solution.csv')

In [3]:
train.head()

,id,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,1,Vistara,UK-810,Bangalore,Early_Morning,one,Night,Mumbai,Economy,14.25,21,7212
1,2,SpiceJet,SG-5094,Hyderabad,Evening,zero,Night,Kolkata,Economy,1.75,7,5292
2,3,Vistara,UK-846,Bangalore,Morning,one,Evening,Delhi,Business,9.58,5,60553
3,4,Vistara,UK-706,Kolkata,Morning,one,Evening,Hyderabad,Economy,6.75,28,5760
4,5,Indigo,6E-5394,Chennai,Early_Morning,zero,Morning,Mumbai,Economy,2.00,4,10712


In [4]:
train['class'] = train['class'].map({'Economy': 0, 'Business': 1})
train['stops'] = train['stops'].map({'one': 1, 'zero': 0, 'two_or_more': 2})

In [5]:
# o'rtacha qimmat va o'rtacha arzon aviakompaniyalar
train.groupby(['airline']).agg({'price': 'mean'})
# ['AirAsia', 'Indigo', 'GO_FIRST','SpiceJet','Air_India', 'Vistara']

,price
airline,
AirAsia,4149.467803
Air_India,23622.449637
GO_FIRST,5683.032493
Indigo,5339.213748
SpiceJet,6059.975248
Vistara,30473.743409


In [6]:
train['airline'] = train['airline'].map({'AirAsia': 0, 'Indigo': 1, 'GO_FIRST': 2, 'SpiceJet': 3, 'Air_India': 4, 'Vistara': 5})
train.head()

,id,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,1,5,UK-810,Bangalore,Early_Morning,1,Night,Mumbai,0,14.25,21,7212
1,2,3,SG-5094,Hyderabad,Evening,0,Night,Kolkata,0,1.75,7,5292
2,3,5,UK-846,Bangalore,Morning,1,Evening,Delhi,1,9.58,5,60553
3,4,5,UK-706,Kolkata,Morning,1,Evening,Hyderabad,0,6.75,28,5760
4,5,1,6E-5394,Chennai,Early_Morning,0,Morning,Mumbai,0,2.00,4,10712


In [7]:
print(train.groupby(['source_city']).agg({'price': 'max'}))
print(train.groupby(['source_city']).agg({'price': 'mean'}))

              price
source_city        
Bangalore     93655
Chennai      105744
Delhi        114434
Hyderabad     97045
Kolkata      106753
Mumbai       114523
                    price
source_city              
Bangalore    21801.289443
Chennai      21488.015480
Delhi        19070.602685
Hyderabad    19199.951237
Kolkata      21451.463777
Mumbai       22588.483530


In [8]:
# print(train.groupby(['arrival_time']).agg({'price': 'max'}))
# print(train.groupby(['arrival_time']).agg({'price': 'mean'}))

In [9]:
train.groupby(['arrival_time']).mean('mean')

,id,airline,stops,class,duration,days_left,price
arrival_time,,,,,,,
Afternoon,10114.815457,3.325315,0.883675,0.273659,11.312390,26.022476,18033.166009
Early_Morning,10049.473108,2.983068,0.901394,0.208167,12.428277,27.821713,13961.313745
Evening,9986.094808,3.780474,0.964823,0.342175,12.523328,25.505643,23375.381866
Late_Night,9754.351528,1.912664,0.902838,0.122271,8.819203,27.521834,10841.620087
Morning,9930.723691,3.921439,0.890847,0.364905,14.063395,25.880323,22943.914097
Night,10040.891241,3.682351,0.929990,0.312927,11.441044,25.726473,21412.093618


In [10]:
train.select_dtypes(include=(int, float)).corr().abs()
# train.hist(bins=50, figsize=(12,8))
# sns.countplot(train)

,id,airline,stops,class,duration,days_left,price
id,1.000000,0.000096,0.006179,0.000039,0.000458,0.003195,0.002443
airline,0.000096,1.000000,0.127579,0.426693,0.369607,0.039084,0.455374
stops,0.006179,0.127579,1.000000,0.000567,0.470493,0.003238,0.121455
class,0.000039,0.426693,0.000567,1.000000,0.144831,0.022332,0.937868
duration,0.000458,0.369607,0.470493,0.144831,1.000000,0.020091,0.213158
days_left,0.003195,0.039084,0.003238,0.022332,0.020091,1.000000,0.102545
price,0.002443,0.455374,0.121455,0.937868,0.213158,0.102545,1.000000


In [11]:
train.isnull().sum()

id                  0
airline             0
flight              0
source_city         0
departure_time      0
stops               0
arrival_time        0
destination_city    0
class               0
duration            0
days_left           0
price               0
dtype: int64

In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                20000 non-null  int64  
 1   airline           20000 non-null  int64  
 2   flight            20000 non-null  object 
 3   source_city       20000 non-null  object 
 4   departure_time    20000 non-null  object 
 5   stops             20000 non-null  int64  
 6   arrival_time      20000 non-null  object 
 7   destination_city  20000 non-null  object 
 8   class             20000 non-null  int64  
 9   duration          20000 non-null  float64
 10  days_left         20000 non-null  int64  
 11  price             20000 non-null  int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 1.8+ MB


In [13]:
train.shape

(20000, 12)

In [14]:
train.select_dtypes(include='object').columns

Index(['flight', 'source_city', 'departure_time', 'arrival_time',
       'destination_city'],
      dtype='object')

In [15]:
# from sklearn.preprocessing import OneHotEncoder

# encoder = OneHotEncoder()

# train_cat_1hot = encoder.fit_transform(train[['airline', 'flight', 'source_city', 'departure_time', 'stops', 'arrival_time', 'destination_city', 'class']]).toarray()

In [16]:
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# train_nums = scaler.fit_transform(train[['duration', 'days_left']])

# Machine Learningga tayyorlaymiz

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

X = train.drop('price', axis=1)
y = train['price']

scaler = StandardScaler()

nums = ['duration', 'days_left']
cats = ['airline', 'flight', 'source_city', 'departure_time', 'stops', 'arrival_time', 'destination_city', 'class']

num_transformer = StandardScaler()
cat_transformer = OneHotEncoder(handle_unknown='ignore')


full_pipeline = ColumnTransformer(
    transformers=[
        ('num', num_transformer, nums),  # Raqamli ustunlarni o'zgartirish
        ('cat', cat_transformer, cats)   # Kategorik ustunlarni o'zgartirish
    ])

# df = pd.get_dummies(df, columns=cats)
X_prepared = full_pipeline.fit_transform(X)

X_train, X_valid, y_train, y_valid = train_test_split(X_prepared, y, test_size=0.2, random_state=42)

X_train.shape

(16000, 1347)

# Baholash funksiyamiz

In [18]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score

def score(y_train, y_pred):
    mae = mean_absolute_error(y_train, y_pred)
    mse = mean_squared_error(y_train, y_pred)
    r2 = r2_score(y_train, y_pred)
    # EVS hisoblash
    evs = explained_variance_score(y_train, y_pred)
    print(f'MAE: {mae}')
    print(f'RMSE: {np.sqrt(mse)}')
    print(f'R-squared: {r2}')
    print(f"Explained Variance Score: {evs}")

# Bir nechta regressiya algoritmlari bilan model yozamiz

In [19]:
# LinearRegression
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression().fit(X_train, y_train)
lr_y_pred = lr_model.predict(X_valid)

print('LinearRegression (valid set)\n')
score(y_valid, lr_y_pred)

LinearRegression (valid set)

MAE: 4376.560081297758
RMSE: 6334.58799956947
R-squared: 0.9235844820919388
Explained Variance Score: 0.9236273011007359


In [20]:
# RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor().fit(X_train, y_train)
rf_y_pred = rf_model.predict(X_valid)

print('RandomForestRegressor:\n')
score(y_valid, rf_y_pred)

RandomForestRegressor:

MAE: 1595.7914185000002
RMSE: 3405.0336958018047
R-squared: 0.9779205809827086
Explained Variance Score: 0.9779228549308304


In [21]:
# DecisionTreeRegressor
from sklearn.tree import DecisionTreeRegressor

tree_model = DecisionTreeRegressor().fit(X_train, y_train)
tree_y_pred = tree_model.predict(X_valid)

print('RandomForestRegressor:\n')
score(y_valid, tree_y_pred)

RandomForestRegressor:

MAE: 1777.541375
RMSE: 4378.637269323927
R-squared: 0.963489082045988
Explained Variance Score: 0.9635079092206706


# Test data bilan predict qilamiz

In [22]:
test_prepared = full_pipeline.transform(test)
test_prepared

<5000x1347 sparse matrix of type '<class 'numpy.float64'>'
	with 34957 stored elements in Compressed Sparse Row format>

In [23]:
y_pred = rf_model.predict(test_prepared)
y_pred

array([39593.02, 33450.17, 14242.86, ..., 34183.21, 31717.77, 31970.63])

# Natijani sample datasetining price ustuniga saqlaymiz

In [24]:
sample['price'] = y_pred

In [25]:
sample.head()

,id,price
0,1,39593.02
1,2,33450.17
2,3,14242.86
3,4,28235.72
4,5,32273.12


In [26]:
sample.to_csv('aviachiptabashorat.csv', index=False)